In [ ]:
!pip install pycocotools
import pycocotools

In [ ]:
import os
!mkdir /kaggle/working/pytorch_od
os.chdir("/kaggle/working/pytorch_od")

In [ ]:
# !rm -r /kaggle/working/pytorch_od/*

In [ ]:
%%bash
git clone https://github.com/pytorch/vision.git
cd vision
git checkout -f v0.3.0
cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

In [ ]:
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import pandas as pd
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from engine import train_one_epoch, evaluate
import utils
import transforms as T

In [ ]:
DATA_PATH = '/kaggle/input/tensorflow-great-barrier-reef'
images_path = os.path.join(DATA_PATH,'train_images')
df_test = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/test.csv")
df_train = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/train.csv")
sample_submission = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/example_sample_submission.csv")
example = np.load("/kaggle/input/tensorflow-great-barrier-reef/example_test.npy")

In [ ]:
import ast
df_train['img_path'] = os.path.join('../input/tensorflow-great-barrier-reef/train_images')+"/video_"+df_train.video_id.astype(str)+"/"+df_train.video_frame.astype(str)+".jpg"
df_train['annotations'] = df_train['annotations'].apply(lambda x: ast.literal_eval(x))
df_train['Number_bbox'] = df_train['annotations'].apply(lambda x:len(x)) 

In [ ]:
def bbox_areas(annotations):
    if not annotations:
        return [0]
    area_list = []
    for annotation in annotations:
        area_list.append(annotation['width']*annotation['height'])
    return area_list
df_train["bbox_area"] = df_train["annotations"].apply(bbox_areas)
df_train["max_area"] = df_train["bbox_area"].apply(lambda x : max(x))
df_train["min_area"] = df_train["bbox_area"].apply(lambda x : min(x))

In [ ]:


def img_viz(df_train, id):
    image = df_train['img_path'][id]
    img = Image.open(image)
    
    for box in df_train['annotations'][id]:
        shape = [box['x'], box['y'], box['x']+box['width'], box['y']+box['height']]
        ImageDraw.Draw(img).rectangle(shape, outline ="red", width=3)
    display(img)
df_train.sort_values("max_area", ascending=False).head()



In [ ]:
train_data = df_train.query("Number_bbox>0")
len(train_data)


In [ ]:
train_data['class'] = 'starfish'
train_data['filename'] = train_data.apply(lambda x: x['img_path'].split('/')[-1], axis=1)

In [ ]:
train_data = train_data.explode("annotations").reset_index()
train_data = train_data.drop(['index'],axis=1)



In [ ]:
train_data

In [ ]:
def get_bbox_dim(annotation):
#     print(annotation)
    return [annotation['x'], annotation['y'],annotation['x'] + annotation['width'],annotation['y'] + annotation['height'],  annotation['width'],annotation['height'] ]



In [ ]:
annotations_data = train_data.apply(lambda row: get_bbox_dim(row["annotations"]), axis=1)
annotations_df = pd.DataFrame(list(annotations_data), columns=['xmin', 'ymin', 'xmax', 'ymax', 'width', 'height'])

In [ ]:
from PIL import Image

train_data = pd.concat([annotations_df, train_data], axis=1)
xmax = np.array(train_data["xmax"].values.tolist())
ymax = np.array(train_data["ymax"].values.tolist())
train_data["xmax"] = np.where(xmax > 1280, 1280, xmax).tolist()
train_data["ymax"] = np.where(ymax > 720, 720, ymax).tolist()





In [ ]:
train_data_video1 = train_data[train_data['video_id']==1].reset_index()



In [ ]:
train_data_video1.to_csv('/kaggle/working/train_data.csv')

In [ ]:
# COLUMNS_TO_KEEP = ['width', 'height', 'xmin', 'ymin', 'xmax', 'ymax', 'filename','class']
# train_csv = train_data_video1[COLUMNS_TO_KEEP]





In [ ]:
def parse_one_annot(path_to_data_file, filename):
    data = pd.read_csv(path_to_data_file)
    boxes_array = data[data["filename"] == filename][["xmin", "ymin","xmax", "ymax"]].values
    return boxes_array

In [ ]:
class GreatReefDataset(torch.utils.data.Dataset):
    def __init__(self, root, data_file, transforms=None, data_df=None):
        self.root = root
        self.transforms = transforms
        if data_df:
            self.data_df = data_df
        else:
            self.data_df = pd.read_csv(data_file)
        self.imgs = list(self.data_df['filename'])
        self.path_to_data_file = data_file
      
    def __getitem__(self, idx):
        img_path = os.path.join(self.root, self.imgs[idx])
        img = Image.open(img_path).convert("RGB")
        box_list = parse_one_annot(self.path_to_data_file,self.imgs[idx])
#         print(box_list)
        boxes = torch.as_tensor(box_list, dtype=torch.float32)
        num_objs = len(box_list)
      # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:,0])
      # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        
        if self.transforms is not None:
           img, target = self.transforms(img, target)
        return img, target

    def __len__(self):
        return len(self.imgs)

In [ ]:
dataset = GreatReefDataset(root= "/kaggle/input/tensorflow-great-barrier-reef/train_images/video_1/",
                         data_file= "/kaggle/working/train_data.csv")
test = dataset.__getitem__(0)[1]

In [ ]:
import torchvision
def get_model(num_classes):
   # load an object detection model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)# get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
   # replace the pre-trained head with a new on
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes)
   
    return model

In [ ]:
def get_transform(train):
    transforms = []
   # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
      # during training, randomly flip the training images
      # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
dataset = GreatReefDataset(root= "/kaggle/input/tensorflow-great-barrier-reef/train_images/video_1/",
                         data_file= "/kaggle/working/train_data.csv",transforms = get_transform(train=True))
dataset_test = GreatReefDataset(root= "/kaggle/input/tensorflow-great-barrier-reef/train_images/video_1/",
                         data_file= "/kaggle/working/train_data.csv",transforms = get_transform(train=False))# split the dataset in train and test settorch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-400])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-400:])# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
              dataset, batch_size=8, shuffle=True, num_workers=4,
              collate_fn=utils.collate_fn)
data_loader_test = torch.utils.data.DataLoader(
         dataset_test, batch_size=1, shuffle=False, num_workers=4,
         collate_fn=utils.collate_fn)
print("We have: {} examples, {} are training and {} testing".format(len(indices), len(dataset), len(dataset_test)))

In [ ]:
torch.cuda.is_available()

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')# our dataset has two classes only - raccoon and not racoon
num_classes = 2
# get the model using our helper function
model = get_model(num_classes)
# move model to the right device
model.to(device)# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)# and a learning rate scheduler which decreases the learning rate by # 10x every 3 epochslr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [ ]:
num_epochs = 1
for epoch in range(num_epochs):
   # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch,
                   print_freq=10)# update the learning rate
    lr_scheduler.step()
   # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

In [ ]:
os.mkdir("/kaggle/working/saved_models")
torch.save(model.state_dict(), "/kaggle/working/saved_models/faster_rcnn")

In [ ]:
loaded_model = get_model(num_classes = 2)
loaded_model.load_state_dict(torch.load( "/kaggle/working/saved_models/faster_rcnn"))

In [ ]:
from PIL import Image, ImageDraw
idx = 0
img, _ = dataset_test[idx]
label_boxes = np.array(dataset_test[idx][1]["boxes"])#put the model in evaluation mode
loaded_model.eval()
with torch.no_grad():
    prediction = loaded_model([img])
    image = Image.fromarray(img.mul(255).permute(1, 2,0).byte().numpy())
    draw = ImageDraw.Draw(image)# draw groundtruth
    print(prediction)
    for elem in range(len(label_boxes)):
        draw.rectangle([(label_boxes[elem][0], label_boxes[elem][1]),
       (label_boxes[elem][2], label_boxes[elem][3])], 
        outline ="green", width =3)
    for element in range(len(prediction[0]["boxes"])):
        boxes = prediction[0]["boxes"][element].cpu().numpy()
        score = np.round(prediction[0]["scores"][element].cpu().numpy(),
                        decimals= 4)
        if score > 0.8:
            draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])], 
            outline ="red", width =3)
            draw.text((boxes[0], boxes[1]), text = str(score))
    display(image)

In [ ]:
import os
!pwd
os.chdir("/")
# ../input/tensorflow-great-barrier-reef/train_images
os.listdir("/kaggle/input")

In [ ]:
df_test

In [ ]:
df_train.head(3)

In [ ]:
sample_submission

In [ ]:
import greatbarrierreef
# env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      #

In [ ]:
from PIL import Image, ImageDraw
idx = 0
img, _ = dataset_test[idx]
label_boxes = np.array(dataset_test[idx][1]["boxes"])#put the model in evaluation mode
loaded_model.eval()
with torch.no_grad():
    prediction = loaded_model([img])
    print(prediction)
    display(image)

In [ ]:
def predict(img):
    prediction = loaded_model([img])
    return prediction


In [ ]:
predict(img)

In [ ]:
img, pred_df = (iter_test)
print(pred_df)

In [ ]:
env.predict(img)

In [ ]:
for idx, (img, pred_df) in enumerate(iter_test):
    print(idx)
    display(img)
    prediction = predict(img)
    print(prediction)
    break